In [1]:
import time
import pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from utils import load_corpus, data_suffle

stopwordPath = './data/stopword.txt'
userDictPath = './data/user_dict.txt'
csvFilePath = '../../corpus/100k/all.csv'
modelPath = './data/bayes.model'

# 载入自定义字典
# jieba.load_userdict(userDictPath)

time_start = time.time()

labels, reviews = load_corpus(csvFilePath, stopwordPath)
labels, reviews = data_suffle(labels, reviews)
# 将reviews的格式转为[str]，为CountVectorizer使用
reviews = [' '.join(review) for review in reviews]

# 1/4 分割数据集
n = len(labels) // 5
labels_train, reviews_train = labels[n:], reviews[n:]
labels_test, reviews_test = labels[:n], reviews[:n]

print(f'Load Corpus Cost {time.time() - time_start:.4f} Sec')

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/3q/s6tg6hsj09x1mhcxccvz_nbw0000gn/T/jieba.cache
Loading model cost 0.441 seconds.
Prefix dict has been built successfully.


Load Corpus Cost 54.0893 Sec


In [22]:
time_start = time.time()

# 加载bayes分类器
# 统计法向量化
vectorizer = CountVectorizer(max_df=0.8, min_df=5)
tfidftransformer = TfidfTransformer()
# 先转换成词频矩阵，再计算TFIDF值
tfidf = tfidftransformer.fit_transform(vectorizer.fit_transform(reviews_train))
# 朴素贝叶斯中的多项式分类器
clf = MultinomialNB().fit(tfidf, labels_train)

print(f'Train Model Cost {time.time() - time_start:.4f} Sec')

Train Model Cost 0.9763 Sec


In [23]:
# 生成测试数据的tfidf矩阵
tfidf_test = tfidftransformer.transform(vectorizer.transform(reviews_test))

In [24]:
tfidf_test.shape, tfidf.shape

((23997, 23163), (95991, 23163))

In [25]:
result = clf.predict(tfidf_test)

In [26]:
from sklearn import metrics
print(metrics.classification_report(labels_test, result))
print("准确率:", metrics.accuracy_score(labels_test, result))

              precision    recall  f1-score   support

           0       0.84      0.78      0.81     12081
           1       0.79      0.85      0.82     11916

    accuracy                           0.81     23997
   macro avg       0.81      0.81      0.81     23997
weighted avg       0.81      0.81      0.81     23997

准确率: 0.8109763720465059


In [31]:
for index in range(100):
    if labels_test[index] != result[index]:
        print(labels_test[index], result[index])
        print(reviews_test[index])

1 0
最近 总是 看到 一句 话 说 不 愿意 公开 秀 恩爱 的 女人 都 是 有 备胎 不 愿意 公开 秀 恩爱 的 男人 都 是 在 给 婊子 留 机会 秀 [嘻嘻] [嘻嘻] [嘻嘻]
1 0
数字 时代 的 低级 错误 常见 愧疚 [哈哈] 转发 微博
0 1
我要 喝 豆浆 [鼓掌] 竞争 太 激烈 [抓狂] [抓狂] 吃喝玩乐 大 搜索 的 娘家 人们 行动 美食 爱好者 滴 福音
0 1
[衰]
1 0
看到 小蕙 的 笔记本 泪流满面 [泪] 妈妈 以后 一定 多爱 一点 [爱你]
1 0
回复 是 的 在 悉尼 在 么 回复 明天 早起 [嘻嘻] [    ] 明天 不想 早起
0 1
夏大 小姐 又 取笑 [晕] 哈哈哈 这 是 的 心结 [哈哈] [哈哈] [哈哈] 瞧瞧 得色 得 [酷] 哈哈
0 1
排队 买 稻香村 糕点 带回家 同样 是 女人 看看 这 姐妹 儿 的 待遇 [泪] [泪] [泪]
0 1
高级 定制 彩宝 系列 超 美的 紫晶 戒指 [色] [色] [色] 增光添彩 纯正 的 颜色 完美 的 净度 切割 在 搭配 柔美 的 玫瑰 金 真是 美死 [抓狂] [抓狂] [抓狂] 还要 感谢 某某 惊爪 爪 童鞋 超 美的 秀 [偷笑] 喜欢 的 快快 举手 [得意] [胜利]
0 1
知道 吗 想 回复 说 的 太 对了 应该 艾特沁沁 字 打 一半 送 外卖 的 来 敲门 耽误 的 评论 我俩 想法 相当 默契 详见 这 一篇 后面 偶俩 对话 [衰] [衰] [衰] 喵 儿
1 0
进口 空气 能 大发 一笔 [哈哈] 新鲜空气
0 1
小伙 挺帅 回复 小东 老师 来来 [哼] 不 来 你拉上 王小东 三小 一起 比个 [怒]
0 1
回复 对 吓 吓 [偷乐] 回复 嘘 鬼爷 吓 吓 [太开心] [太开心] [太开心] 回复 可笑 死 老 鬼 哈哈哈哈 哈哈哈 这 小调 皮儿 [太开心] [可爱] [泪] [泪] [泪] [泪] 请 吃 汤饭 鬼爷 事 的 [泪]
1 0
[可怜] 木 有人 呀 [抓狂] 打算 对 谁 表白 呢 [亲亲] 很想 参与 一下 [哈哈]
0 1
[吃惊] 都 要学 表演 [泪] 坐车去 咯 [嘻嘻] [泪] 都 得 一笑 [哈哈] [嘻嘻] 嚎啕大哭
0 1
大 娃娃 亲爱 的 大 